# ETL
### El siguiente ETL detallará los hallazgos y soluciones para los requerimientos de tranformación de datos detallados a continuación:

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API

2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

3. Los valores nulos del campo release date deben eliminarse.

4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [1]:
# Se importan las bibliotecas necesarias
import pandas as pd
import json
import ast
import numpy as np
import re

In [2]:
# Se ingestan los datos
df_movies = pd.read_csv('data_set/movies_dataset.csv')
df_credits = pd.read_csv('data_set/credits.csv')

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\1939279161.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('data_set/movies_dataset.csv')


### Exploración de los dos datasets a transformar

In [3]:
# Se visualiza el resumen del primer data set
print (df_movies.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [4]:
# Se visualiza el resumen del segundo data set
print (df_credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


### Preparación de los dataframes previamente a las transformaciones requeridas

In [5]:
''' Unión de los datasets '''

# Primeramente se realizará el merge para luego transformar los datos usando un solo dataset 
# Pero antes se requiere igualar sus id a 'int'
df_movies['id'].isnull().sum()  # Hay 3 valores nulos, lo cual evitará realizar la conversion a 'int'
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce') # Se forza la conversion de 'object' a 'float'
df_movies = df_movies.dropna(subset=['id']) # Se procede a eliminar los tres valores nulos
df_movies['id'] = df_movies['id'].astype(int) #Se realiza la conversion de 'float' a 'int'
df_merge = pd.merge(df_movies, df_credits, on='id', how='inner') # Finalmente se hace la union de los datasets

In [6]:
# Se visualiza que hay 2 columnas añadidas provenientes del segundo dataset. Tenemos en total 26 columnas
print (df_merge.info())
print (df_merge.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45538 entries, 0 to 45537
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45538 non-null  object 
 1   belongs_to_collection  4500 non-null   object 
 2   budget                 45538 non-null  object 
 3   genres                 45538 non-null  object 
 4   homepage               7792 non-null   object 
 5   id                     45538 non-null  int32  
 6   imdb_id                45521 non-null  object 
 7   original_language      45527 non-null  object 
 8   original_title         45538 non-null  object 
 9   overview               44584 non-null  object 
 10  popularity             45535 non-null  object 
 11  poster_path            45152 non-null  object 
 12  production_companies   45535 non-null  object 
 13  production_countries   45535 non-null  object 
 14  release_date           45451 non-null  object 
 15  re

In [7]:
# Para efectos de practicidad se cambia el nombre de nuestro dataframe a df
df = df_merge

In [8]:

# Se visualiza la estructura de anidación de cada columna
# Se halla que solamente las columnas 'name' anidadas son reelevantes para efectos de consulta
# Se halla tambien que los datos anidados tienen diferente estructura que se detalla a continuación:

print (df.iloc[0]['belongs_to_collection'])  # 'Diccionario'
print (df.iloc[0]['genres'])   # 'Listas de Diccionarios o json'
print (df.iloc[0]['spoken_languages'])   # 'Listas de Diccionarios o json'
print (df.iloc[0]['production_companies'])  # 'Listas de Diccionarios o json'
print (df.iloc[0]['production_countries'])  # 'Listas de Diccionarios o json'
print (df.iloc[0]['cast']) # 'Listas de Diccionarios o json'
print (df.iloc[0]['crew']) # 'Listas de Diccionarios o json'

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}
[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
[{'iso_639_1': 'en', 'name': 'English'}]
[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'iso_3166_1': 'US', 'name': 'United States of America'}]
[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSd

### 1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API

In [9]:
# Se crea una clase en donde se convierte a diccionario los datos anidados y luego se procede a extraer
# los valores de las llaves 'name' de cada columna. Segun

class DataTransformer:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def convertir_a_dicc(self, column):
        return column.apply(lambda x: ast.literal_eval(x) if pd.notna(x) else np.nan)

    def desanidar_columna(self, column):
        return column.apply(lambda x: ', '.join([d['name'] for d in x]) if isinstance(x, list) else np.nan)

    def convertir_a_dicc_btc(self, column):
        return column.apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))

    def desanidar_btc(self, column):
        return column.apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else np.nan)

    def transform(self):
        self.dataframe['genres'] = self.convertir_a_dicc(self.dataframe['genres'])
        self.dataframe['spoken_languages'] = self.convertir_a_dicc(self.dataframe['spoken_languages'])
        self.dataframe['production_companies'] = self.convertir_a_dicc(self.dataframe['production_companies'])
        self.dataframe['production_countries'] = self.convertir_a_dicc(self.dataframe['production_countries'])
        self.dataframe['cast'] = self.convertir_a_dicc(self.dataframe['cast'])
        self.dataframe['crew'] = self.convertir_a_dicc(self.dataframe['crew'])
        self.dataframe['belongs_to_collection'] = self.convertir_a_dicc_btc(self.dataframe['belongs_to_collection'])
        
        self.dataframe['genres'] = self.desanidar_columna(self.dataframe['genres'])
        self.dataframe['spoken_languages'] = self.desanidar_columna(self.dataframe['spoken_languages'])
        self.dataframe['production_companies'] = self.desanidar_columna(self.dataframe['production_companies'])
        self.dataframe['production_countries'] = self.desanidar_columna(self.dataframe['production_countries'])
        self.dataframe['cast'] = self.desanidar_columna(self.dataframe['cast'])
        self.dataframe['crew'] = self.desanidar_columna(self.dataframe['crew'])
        self.dataframe['belongs_to_collection'] = self.desanidar_btc(self.dataframe['belongs_to_collection'])
        
        return self.dataframe


def convert_and_desanidate_dataframe(dataframe):
    data_transformer = DataTransformer(dataframe)
    return data_transformer.transform()




In [10]:
# Se llama a la función convert_and_desanidate_dataframe desde fuera de la clase
transformed_df = convert_and_desanidate_dataframe(df)

In [11]:
# Se verifica que se han extraido bien los valores de las claves 'name'
print (df.iloc[0]['belongs_to_collection'])  # 'Diccionario'
print (df.iloc[0]['genres'])   # 'Listas de Diccionarios o json'
print (df.iloc[0]['spoken_languages'])   # 'Listas de Diccionarios o json'
print (df.iloc[0]['production_companies'])  # 'Listas de Diccionarios o json'
print (df.iloc[0]['production_countries'])  # 'Listas de Diccionarios o json'
print (df.iloc[0]['cast']) # 'Listas de Diccionarios o json'
print (df.iloc[0]['crew']) # 'Listas de Diccionarios o json'

Toy Story Collection
Animation, Comedy, Family
English
Pixar Animation Studios
United States of America
Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, John Morris, Erik von Detten, Laurie Metcalf, R. Lee Ermey, Sarah Freeman, Penn Jillette
John Lasseter, Joss Whedon, Andrew Stanton, Joel Cohen, Alec Sokolow, Bonnie Arnold, Ed Catmull, Ralph Guggenheim, Steve Jobs, Lee Unkrich, Ralph Eggleston, Robert Gordon, Mary Helen Leasman, Kim Blanchette, Marilyn McCoppen, Randy Newman, Dale E. Grahn, Robin Cooper, John Lasseter, Pete Docter, Joe Ranft, Patsy Bouge, Norm DeCarlo, Ash Brannon, Randy Newman, Roman Figun, Don Davis, James Flamberg, Mary Beth Smith, Rick Mackay, Susan Bradley, William Reeves, Randy Newman, Andrew Stanton, Pete Docter, Gary Rydstrom, Karen Robert Jackson, Chris Montan, Rich Quade, Michael Berenstein, Colin Brady, Davey Crockett Feiten, Angie Glocka, Rex Grignon, Tom K. Gurney, Jimmy Hayward, Hal T. Hickel, Karen Kiser, Ant

  ### 2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [12]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

### 3. Los valores nulos del campo release date deben eliminarse

In [13]:
df = df.dropna(subset = ['release_date'])

### 4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [14]:
# Se vizualiza el formato de la columna
print (df['release_date'].iloc[0])
print (df['release_date'].dtype)

1995-10-30
object


In [15]:

# Se cambia el tipo de dato a datetime y se le da formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\2444850055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')


In [16]:
# Finalmente se procede a crear una nueva columna llamada release_year que contenga el año de release_date
df['release_year'] = df['release_date'].dt.year

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\929720481.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_year'] = df['release_date'].dt.year


### 5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [17]:
# Se revisa los datos y se encuentra que ambas columnas son de tipo float, pero es innecesario al no poseer
# valores decimales. Por eso se procede a convertir a enteros, no sin antes convertir NAN a ceros
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\3919497758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revenue'] = df['revenue'].fillna(0)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\3919497758.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['budget'] = df['budget'].fillna(0)


In [18]:
df['revenue'] = df['revenue'].astype(int) #Se realiza la conversion de 'float' a 'int'
df['budget'] = df['budget'].astype(int) #Se realiza la conversion de 'float' a 'int'

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\1443137782.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revenue'] = df['revenue'].astype(int) #Se realiza la conversion de 'float' a 'int'
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\1443137782.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['budget'] = df['budget'].astype(int) #Se realiza la conversion de 'float' a 'int'


In [19]:
df['return'] = df['revenue'].div(df['budget'], fill_value=0).round(2).fillna(0)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_22664\3006216347.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['return'] = df['revenue'].div(df['budget'], fill_value=0).round(2).fillna(0)


### 6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [20]:
df = df.drop(columns = ['video','imdb_id','adult','original_title','poster_path','homepage'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45451 entries, 0 to 45537
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4497 non-null   object        
 1   budget                 45451 non-null  int32         
 2   genres                 45451 non-null  object        
 3   id                     45451 non-null  int32         
 4   original_language      45440 non-null  object        
 5   overview               44510 non-null  object        
 6   popularity             45451 non-null  object        
 7   production_companies   45451 non-null  object        
 8   production_countries   45451 non-null  object        
 9   release_date           45451 non-null  datetime64[ns]
 10  revenue                45451 non-null  int32         
 11  runtime                45205 non-null  float64       
 12  spoken_languages       45451 non-null  object        
 13  status

In [22]:
df['return']

0        12.45
1         4.04
2         0.00
3         5.09
4          inf
         ...  
45532     0.00
45534     0.00
45535     0.00
45536     0.00
45537     0.00
Name: return, Length: 45451, dtype: float64

In [23]:
df.iloc[9]

belongs_to_collection                                James Bond Collection
budget                                                            58000000
genres                                         Adventure, Action, Thriller
id                                                                     710
original_language                                                       en
overview                 James Bond must unmask the mysterious head of ...
popularity                                                       14.686036
production_companies                       United Artists, Eon Productions
production_countries              United Kingdom, United States of America
release_date                                           1995-11-16 00:00:00
revenue                                                          352194034
runtime                                                              130.0
spoken_languages                                 English, Pусский, Español
status                   

In [24]:
df.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew', 'release_year', 'return'],
      dtype='object')

In [25]:
df['runtime'].isnull().sum()

246

In [26]:
df['runtime'] = df['runtime'].fillna(0)
df['runtime'] = df['runtime'].astype(int)

In [27]:
df['belongs_to_collection'].isnull().sum()
df['belongs_to_collection'].astype(str)

0                  Toy Story Collection
1                                   nan
2             Grumpy Old Men Collection
3                                   nan
4        Father of the Bride Collection
                      ...              
45532                               nan
45534                               nan
45535                               nan
45536                               nan
45537                               nan
Name: belongs_to_collection, Length: 45451, dtype: object

In [28]:
df['crew'] = df['crew'].astype(str)


In [29]:
df['crew'].iloc[3]

'Forest Whitaker, Ronald Bass, Ronald Bass, Ezra Swerdlow, Deborah Schindler, Terry McMillan, Terry McMillan, Terry McMillan, Kenneth Edmonds, Caron K'

In [30]:
df['director'] = df['crew'].str.split(',').str[0].str.strip()


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45451 entries, 0 to 45537
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4497 non-null   object        
 1   budget                 45451 non-null  int32         
 2   genres                 45451 non-null  object        
 3   id                     45451 non-null  int32         
 4   original_language      45440 non-null  object        
 5   overview               44510 non-null  object        
 6   popularity             45451 non-null  object        
 7   production_companies   45451 non-null  object        
 8   production_countries   45451 non-null  object        
 9   release_date           45451 non-null  datetime64[ns]
 10  revenue                45451 non-null  int32         
 11  runtime                45451 non-null  int32         
 12  spoken_languages       45451 non-null  object        
 13  status

In [32]:
#Exportamos el dataframe limpio
df.to_csv('data_set/ds_clean.csv', index=False)